In [1]:
import sys
sys.path.append('../settings')
from setting import *
print(MODE)
assert MODE != 'UPDATE_MODEL', '実行スクリプトが間違っているよ'

RENEWAL_RESULT


In [2]:
# input
file_horse = list(glob.glob(config['SCRAPED_RESULT']))
display(file_horse)
file_race_info = list(glob.glob(config['SCRAPED_INFO']))
display(file_race_info)

['../../data/scraped\\race_result.csv',
 '../../data/scraped\\race_result2.csv',
 '../../data/scraped\\race_result3.csv',
 '../../data/scraped\\race_result4.csv']

['../../data/scraped\\race_info.csv',
 '../../data/scraped\\race_info2.csv',
 '../../data/scraped\\race_info3.csv',
 '../../data/scraped\\race_info4.csv']

In [3]:
# master
file_master = config['HORSE_MASTER']

In [4]:
# output
file_use_id = config['RACE_HORSE_ID']

In [5]:
judge_cols = ['馬名', 'gender', 'born_year']

# 読み込み

In [6]:
df_list = [] 
for file in file_horse:
    print(file)
    df = pd.read_csv(file, usecols=['race_id', '馬名', '性齢'] )
    print(df.shape)
    df_list.append(df)

df_horse = pd.concat(df_list, ignore_index=True)
print(df_horse.shape)
df_horse.head(1)

../../data/scraped\race_result.csv
(35371, 3)
../../data/scraped\race_result2.csv
(183678, 3)
../../data/scraped\race_result3.csv
(102959, 3)
../../data/scraped\race_result4.csv
(81209, 3)
(403217, 3)


,馬名,性齢,race_id
0,マイネルギブソン,牡2,201001020110


In [7]:
df_list = [] 
for file in file_race_info:
    print(file)
    df = pd.read_csv(file, usecols=['race_id', 'race_date'])
    print(df.shape)
    df_list.append(df)

df_info = pd.concat(df_list, ignore_index=True)
print(df_info.shape)
df_info.head(1)

../../data/scraped\race_info.csv
(2509, 2)
../../data/scraped\race_info2.csv
(12645, 2)
../../data/scraped\race_info3.csv
(7170, 2)
../../data/scraped\race_info4.csv
(5826, 2)
(28150, 2)


,race_id,race_date
0,201001020110,2010年9月11日


# 加工

In [8]:
# 性齢
df_horse['gender'] = df_horse['性齢'].str[0]
df_horse['age'] = df_horse['性齢'].str[1:].astype(np.int16)

In [9]:
# 結合
df_horse_merged = df_horse[['race_id', '馬名', 'age', 'gender']].merge(df_info[['race_id', 'race_date']])
df_horse_merged.race_date = pd.to_datetime(df_horse_merged.race_date, format='%Y年%m月%d日')

In [10]:
# 生まれ年
df_horse_merged['born_year'] = df_horse_merged.race_date.dt.year - df_horse_merged.age
df_horse_merged = df_horse_merged.replace('セ', '牡').drop_duplicates(subset=judge_cols)

In [11]:
df_horse_merged.drop(['age', 'race_date'], axis=1, inplace=True)

# 重複削除

In [12]:
print('MODE: ', MODE)
if MODE == "RENEWAL_RESULT":
    df_horse_dupl = df_horse_merged.loc[
        df_horse_merged.duplicated('馬名', keep=False)
    ].sort_values(['馬名', 'born_year'])
    df_horse_merged = df_horse_merged.loc[~df_horse_merged.馬名.isin(df_horse_dupl.馬名)]

    df_horse_dupl['born_year_diff'] = df_horse_dupl.groupby('馬名').born_year.diff()
    print('重複数: ', df_horse_dupl.shape[0])
    if len(df_horse_dupl) > 0:
        assert df_horse_dupl.born_year_diff.min() > 3, '同名の馬の年齢が近すぎません？'
    df_horse_dupl.drop('born_year_diff', axis=1, inplace=True)

    df_horse_master = pd.concat(
        [df_horse_merged, df_horse_dupl],
        ignore_index=True
    ).drop('race_id', axis=1)
    
    print('マスター件数: ', df_horse_master.shape[0])

    df_horse_master.reset_index(drop=True, inplace=True)
    df_horse_master['horse_id'] = df_horse_master.index + 1
    
elif MODE == "SCORING_PREPARATION":
    df_horse_master = pd.read_csv(file_master)
    
    df_horse_tmp = df_horse_merged.replace({'セ':'牡'}).merge(
        df_horse_master,
        how='left',
        on=judge_cols
    )
    
    df_horse_new = df_horse_tmp.loc[df_horse_tmp.horse_id.isnull()]
    print('新しい子の数: ', df_horse_new.shape[0])
    df_horse_new.reset_index(drop=True, inplace=True)
    df_horse_new['horse_id'] = df_horse_master.horse_id.max() + df_horse_new.index + 1
    df_horse_master = pd.concat([df_horse_master, df_horse_new[df_horse_master.columns]])
    assert df_horse_master.shape[0] == df_horse_master.horse_id.nunique(), 'horse_idが間違っている'

MODE:  RENEWAL_RESULT
重複数:  194
マスター件数:  46414


In [13]:
df_horse_out = df_horse_merged.replace({'セ':'牡'}).merge(
    df_horse_master,
    how='left',
    on=['馬名', 'born_year', 'gender']
)
assert df_horse_out.shape[0] == df_horse_merged.shape[0], '結合ミス'
df_horse_out[['race_id', 'horse_id', '馬名']].to_csv(file_use_id, index=False)

In [14]:
df_horse_master.to_csv(file_master, index=False)